## 참조자료

https://hwiyong.tistory.com/105 

In [2]:
from tensorflow import keras
from tensorflow.keras import layers


In [2]:
inputs = keras.Input(shape=(784,), name='digits')

In [3]:

x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, activation='softmax', name='predictions')(x)

In [4]:
model = keras.Model(inputs=inputs, outputs=outputs)

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, 784)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
predictions (Dense)          (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [6]:
# Load a toy dataset for the sake of this example
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Preprocess the data (these are Numpy arrays)
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

## 손실함수를 희소행렬로 처리하기

In [3]:
keras.losses.CategoricalCrossentropy

tensorflow.python.keras.losses.CategoricalCrossentropy

In [7]:
# Specify the training configuration (optimizer, loss, metrics)
model.compile(optimizer=keras.optimizers.RMSprop(),  # Optimizer
              # Loss function to minimize
              loss=keras.losses.SparseCategoricalCrossentropy(),
              # List of metrics to monitor
              metrics=[keras.metrics.SparseCategoricalAccuracy()])

In [ ]:
## 문자열로 지정해도 
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])

## 검증 데이터를 비율로 처리하기

In [8]:
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=3,
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    validation_split= 0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/3
48000/48000 [==============================] - 1s 26us/sample - loss: 0.3561 - sparse_categorical_accuracy: 0.9008 - val_loss: 0.1928 - val_sparse_categorical_accuracy: 0.9449
Epoch 2/3
48000/48000 [==============================] - 1s 20us/sample - loss: 0.1638 - sparse_categorical_accuracy: 0.9509 - val_loss: 0.1401 - val_sparse_categorical_accuracy: 0.9596
Epoch 3/3
48000/48000 [==============================] - 1s 20us/sample - loss: 0.1185 - sparse_categorical_accuracy: 0.9644 - val_loss: 0.1284 - val_sparse_categorical_accuracy: 0.9606


In [10]:
# The returned "history" object holds a record
# of the loss values and metric values during training
print('\nhistory dict:', history.history)




history dict: {'loss': [0.356132064918677, 0.163806191260616, 0.11851078534747163], 'sparse_categorical_accuracy': [0.90077084, 0.9509375, 0.9644167], 'val_loss': [0.192791157801946, 0.14007131407409906, 0.12844272445018093], 'val_sparse_categorical_accuracy': [0.94491667, 0.95958334, 0.9605833]}


In [11]:
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(x_test, y_test, batch_size=128)
print('test loss, test acc:', results)


# Evaluate on test data
10000/1 [======================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

test loss, test acc: [0.12254318649321795, 0.9618]


In [12]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print('\n# Generate predictions for 3 samples')
predictions = model.predict(x_test[:3])
print('predictions shape:', predictions.shape)


# Generate predictions for 3 samples
predictions shape: (3, 10)


## 검증데이터를 분리해서 처리하기 

In [13]:
# Reserve 10,000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

In [14]:
print('# Fit model on training data')
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=3,
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    validation_data=(x_val, y_val))

# Fit model on training data
Train on 50000 samples, validate on 10000 samples
Epoch 1/3
50000/50000 [==============================] - 1s 22us/sample - loss: 0.0958 - sparse_categorical_accuracy: 0.9718 - val_loss: 0.1298 - val_sparse_categorical_accuracy: 0.9632
Epoch 2/3
50000/50000 [==============================] - 1s 19us/sample - loss: 0.0793 - sparse_categorical_accuracy: 0.9766 - val_loss: 0.1062 - val_sparse_categorical_accuracy: 0.9716
Epoch 3/3
50000/50000 [==============================] - 1s 19us/sample - loss: 0.0682 - sparse_categorical_accuracy: 0.9798 - val_loss: 0.1018 - val_sparse_categorical_accuracy: 0.9723


In [15]:
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(x_test, y_test, batch_size=128)
print('test loss, test acc:', results)


# Evaluate on test data
10000/1 [======================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

test loss, test acc: [0.09477657253127546, 0.9723]


In [16]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print('\n# Generate predictions for 3 samples')
predictions = model.predict(x_test[:3])
print('predictions shape:', predictions.shape)


# Generate predictions for 3 samples
predictions shape: (3, 10)


## 문자열로 지정 가능

In [17]:
## 문자열로 지정해도 
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])

In [18]:
print('# Fit model on training data')
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=3,
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    validation_data=(x_val, y_val))

# Fit model on training data
Train on 50000 samples, validate on 10000 samples
Epoch 1/3
50000/50000 [==============================] - 1s 25us/sample - loss: 0.0589 - sparse_categorical_accuracy: 0.9816 - val_loss: 0.1089 - val_sparse_categorical_accuracy: 0.9720
Epoch 2/3
50000/50000 [==============================] - 1s 19us/sample - loss: 0.0518 - sparse_categorical_accuracy: 0.9847 - val_loss: 0.1039 - val_sparse_categorical_accuracy: 0.9731
Epoch 3/3
50000/50000 [==============================] - 1s 19us/sample - loss: 0.0464 - sparse_categorical_accuracy: 0.9856 - val_loss: 0.1292 - val_sparse_categorical_accuracy: 0.9683
